# Code for Hackathon: Schema coverage calculation



Method:

- get unique queries (does not handle the white spaces)
- parse queries
- get normalized queries
- get unique queries from normalized verion and parsed tree(to handle white space problem)
- get triples from queries
- get schema elements from triple pattern
- get schema elements from Knowledge graph
- schema coverage calculation


## 1. We download the query logs from "link to download the logs". 

In [3]:
import pandas as pd

df = pd.read_csv('bio2rdf_sparql_queries.csv', lineterminator='\n', dtype=str, encoding='utf-8')

# Count the number of null values in each column
null_counts = df.isnull().sum()

print("Null value counts:")
print(null_counts)
print(len(df))
pd.set_option('display.max_colwidth', None)  # Display full content of columns
print(df.head())
## Number of queries are 

Null value counts:
query    0
dtype: int64
16
                                                                                                                                  query
0                                                                                                  SELECT * WHERE { ?s ?p ?o } LIMIT 10
1                SELECT ?drug ?label WHERE { ?drug a <http://bio2rdf.org/drugbank_vocabulary:Drug> . ?drug rdfs:label ?label } LIMIT 10
2  SELECT ?gene ?description WHERE { ?gene a <http://bio2rdf.org/gene_vocabulary:Gene> . ?gene rdfs:description ?description } LIMIT 10
3           SELECT ?disease ?name WHERE { ?disease a <http://bio2rdf.org/omim_vocabulary:Disease> . ?disease rdfs:name ?name } LIMIT 10
4        CONSTRUCT { ?s a <http://bio2rdf.org/gene_vocabulary:Gene> } WHERE { ?s a <http://bio2rdf.org/gene_vocabulary:Gene> } LIMIT 10


## 2. We extract unique queries and their count

Note: this code dose not handle whitespaces to identify unique queries. after parsing queries I use parse tree to identify unique queries. current step is only for optimizing query parsing process.

In [4]:
import csv

def extract_unique_queries(input_file, output_file):
    try:
        # Initialize a dictionary to store query counts
        query_counts = {}

        with open(input_file, 'r', newline='\n', encoding='utf-8') as infile:
            reader = csv.DictReader(infile)

            # Iterate through the rows and count queries
            for row in reader:
                query = row['query']  # Assuming 'query' is the correct column name
                query_counts[query] = query_counts.get(query, 0) + 1

        # Sort queries and their counts based on the count column in descending order
        sorted_queries = sorted(query_counts.items(), key=lambda item: item[1], reverse=True)

        # Write sorted queries and their counts to the output CSV file
        with open(output_file, 'w', newline='\n', encoding='utf-8') as outfile:
            writer = csv.writer(outfile, lineterminator='\n')
            writer.writerow(['query', 'count'])  # Write header

            for query, count in sorted_queries:
                writer.writerow([query, count])

        print("Unique queries extracted with counts and written to", output_file)

    except FileNotFoundError:
        print("Error: Input CSV file not found.")
    except Exception as e:
        print("An error occurred:", e)

# Replace 'input_file.csv' with the name of your CSV file and 'output_file.csv' with the desired output filename
extract_unique_queries('bio2rdf_sparql_queries.csv', 'unique_queries_with_counts.csv')


Unique queries extracted with counts and written to unique_queries_with_counts.csv


In [5]:
import pandas as pd

df = pd.read_csv('unique_queries_with_counts.csv', lineterminator='\n', dtype=str)

print(len(df))
df.head(30)

12


,query,count
0,SELECT ?drug ?label WHERE { ?drug a <http://bio2rdf.org/drugbank_vocabulary:Drug> . ?drug rdfs:label ?label } LIMIT 10,4
1,SELECT ?pathway WHERE { ?pathway a <http://bio2rdf.org/kegg_vocabulary:Pathway> . ?pathway <http://bio2rdf.org/kegg_vocabulary:gene> <http://bio2rdf.org/gene:207> } LIMIT 10,2
2,SELECT * WHERE { ?s ?p ?o } LIMIT 10,1
3,SELECT ?gene ?description WHERE { ?gene a <http://bio2rdf.org/gene_vocabulary:Gene> . ?gene rdfs:description ?description } LIMIT 10,1
4,SELECT ?disease ?name WHERE { ?disease a <http://bio2rdf.org/omim_vocabulary:Disease> . ?disease rdfs:name ?name } LIMIT 10,1
5,CONSTRUCT { ?s a <http://bio2rdf.org/gene_vocabulary:Gene> } WHERE { ?s a <http://bio2rdf.org/gene_vocabulary:Gene> } LIMIT 10,1
6,DESCRIBE <http://bio2rdf.org/drugbank:DB00215>,1
7,ASK WHERE { <http://bio2rdf.org/omim:100050> a <http://bio2rdf.org/omim_vocabulary:Disease> },1
8,SELECT ?protein WHERE { ?protein a <http://bio2rdf.org/pro_vocabulary:Protein> . ?protein <http://bio2rdf.org/pro_vocabulary:expressedIn> <http://bio2rdf.org/taxon:9606> } LIMIT 10,1
9,SELECT ?interaction WHERE { ?interaction a <http://bio2rdf.org/drugbank_vocabulary:Drug-Drug-Interaction> . ?interaction <http://bio2rdf.org/drugbank_vocabulary:drug> <http://bio2rdf.org/drugbank:DB00215> } LIMIT 10,1


## 4. Parse the queries:  



- download Node.js from the official website: https://nodejs.org/

- Installed Node js 

- npm install sparqljs


In [6]:

!npm install csv-parser



added 2 packages in 3s

1 package is looking for funding
  run `npm fund` for details


In [7]:
!npm install csv-stringify


added 1 package, and audited 4 packages in 2s

1 package is looking for funding
  run `npm fund` for details

found 0 vulnerabilities


In [8]:
# create a python file with following content(mine is called "optimized_from_js_.py")

import subprocess

def run_script():
    # Run the JavaScript code using Node.js as a subprocess
    result = subprocess.run(['node', 'optimized.js'], stdout=subprocess.PIPE, text=True)

    # Print the output (JSON representation of the parsed SPARQL query)
    print(result.stdout)
    print('hi')

if __name__ == "__main__":
    run_script()



hi


In [ ]:
#  In Unix-like operating systems
!nohup python optimized_from_js_.py 

In [9]:
# In windows
!pythonw optimized_from_js_.py > output90.txt


node:internal/modules/cjs/loader:1080
  throw err;
  ^

Error: Cannot find module 'sparqljs'
Require stack:
- c:\Users\p70073484\Desktop\Hackathon\optimized.js
    at Module._resolveFilename (node:internal/modules/cjs/loader:1077:15)
    at Module._load (node:internal/modules/cjs/loader:922:27)
    at Module.require (node:internal/modules/cjs/loader:1143:19)
    at require (node:internal/modules/cjs/helpers:121:18)
    at Object.<anonymous> (c:\Users\p70073484\Desktop\Hackathon\optimized.js:2:22)
    at Module._compile (node:internal/modules/cjs/loader:1256:14)
    at Module._extensions..js (node:internal/modules/cjs/loader:1310:10)
    at Module.load (node:internal/modules/cjs/loader:1119:32)
    at Module._load (node:internal/modules/cjs/loader:960:12)
    at Function.executeUserEntryPoint [as runMain] (node:internal/modules/run_main:81:12) {
  code: 'MODULE_NOT_FOUND',
  requireStack: [ 'c:\\Users\\p70073484\\Desktop\\Hackathon\\optimized.js' ]
}

Node.js v18.17.1


In [ ]:
import pandas as pd

column_names = ['query', 'parsed_query']

df = pd.read_csv('parsed.csv', lineterminator='\n', dtype=str, header=None, names=column_names)

print(len(df))
df.head(10)

1051679


,query,parsed_query
0,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,"{""queryType"":""ASK"",""where"":[{""type"":""bgp"",""tri..."
1,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,"{""queryType"":""SELECT"",""variables"":[{""termType""..."
2,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,"{""queryType"":""SELECT"",""variables"":[{""termType""..."
3,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,"{""queryType"":""ASK"",""where"":[{""type"":""bgp"",""tri..."
4,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,"{""queryType"":""SELECT"",""variables"":[{""expressio..."
5,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,"{""queryType"":""SELECT"",""variables"":[{""expressio..."
6,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,"{""queryType"":""SELECT"",""variables"":[{""termType""..."
7,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,"{""queryType"":""ASK"",""where"":[{""type"":""bgp"",""tri..."
8,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,"{""queryType"":""SELECT"",""variables"":[{}],""where""..."
9,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,"{""queryType"":""SELECT"",""variables"":[{}],""where""..."


## Following code returns valid queries and their parsed tree(removes unvalid):

In [ ]:
import csv

# Define the input and output file paths
input_file_path = 'parsed.csv'
output_file_path = 'valid_parsed.csv'

# Open the input CSV file for reading
with open(input_file_path, 'r', newline='\n', encoding='utf-8') as input_file:
    # Create a CSV reader without header
    reader = csv.reader(input_file)
    
    # Create a list to store valid rows
    valid_rows = []
    
    for row in reader:
        # Check if the second column is not equal to 'Error parsing the query'
        if len(row) >= 2 and row[1] != 'Error parsing the query.':
            valid_rows.append(row)

# Open the output CSV file for writing
with open(output_file_path, 'w', newline='\n', encoding='utf-8') as output_file:
    # Create a CSV writer with column names
    writer = csv.writer(output_file)
    
    # Write column names
    writer.writerow(['query', 'parsed_query'])
    
    # Write the valid rows
    writer.writerows(valid_rows)

print(f"Valid rows have been written to '{output_file_path}'.")


import pandas as pd

# column_names = ['query', 'parsed_query']

df = pd.read_csv('valid_parsed.csv', lineterminator='\n', dtype=str)

print(len(df))
df.head(10)

In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('valid_parsed.csv', lineterminator='\n', dtype=str)

# Count unique values in the "ColumnName" column
unique_count = df['parsed_query\r'].nunique()

print(f"Number of unique values in the 'parsed_query' column: {unique_count}")


Number of unique values in the 'parsed_query' column: 633453


## Query normalization

Global variations

In [ ]:
import csv

def normalize_sparql_queries(input_csv_path, output_csv_path):
    global_var_counter = 1  # Initialize the global variable counter

    with open(input_csv_path, 'r') as input_csv, open(output_csv_path, 'w', newline='') as output_csv:
        csv_reader = csv.reader(input_csv)
        csv_writer = csv.writer(output_csv)

        for row in csv_reader:
            if not row:
                continue  # Skip empty rows

            sparql_query = row[1]  # Assuming the SPARQL query is in the first column

            # Extract variables from the SPARQL query using regular expressions
            variables = re.findall(r'\?([a-zA-Z_][a-zA-Z0-9_]*)', sparql_query)

            # Create a mapping of old variables to new numerated variables
            variable_mapping = {}
            for variable in variables:
                if variable not in variable_mapping:
                    variable_mapping[variable] = f'?var{global_var_counter}'
                    global_var_counter += 1

            # Replace variables in the SPARQL query with numerated variables
            for old_variable, new_variable in variable_mapping.items():
                sparql_query = sparql_query.replace(f'?{old_variable}', new_variable)

            # Write the normalized query to the output CSV
            csv_writer.writerow([sparql_query])

if __name__ == "__main__":
    input_csv_path = "valid_parsed.csv"  # Replace with the path to your input CSV file
    output_csv_path = "normalized_queries.csv"  # Replace with the desired output CSV file

    normalize_sparql_queries(input_csv_path, output_csv_path)


In [ ]:
import pandas as pd

# Set the maximum column width to None to display the entire content of each cell
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('normalized_queries.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head(10)

get a sample

In [24]:
import pandas as pd

# Define the input and output file paths
input_csv_file = 'find_bug_triples.csv'  # Replace with your input CSV file path
output_csv_file = 't.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_csv_file)

# Get the first 1000 rows, including the header
sample_df = df.head(10 )

# Save the sample DataFrame to an output CSV file
sample_df.to_csv(output_csv_file, index=False)

print(f"{len(sample_df)} rows have been saved to {output_csv_file}.")


10 rows have been saved to t.csv.


# get triples

In [ ]:
import csv
import json

# Load your CSV data from the input file
input_csv_file = 'parsed.csv'
output_csv_file = 'find_bug_triples.csv'

# Create a list to store the triples
triples = []

# Define a function to extract triples from the parse tree
def extract_triples(parse_tree):
    if "triples" in parse_tree:
        for triple in parse_tree["triples"]:
            try:
                subject = triple["subject"]["value"]
                predicate = triple["predicate"]["value"]
                obj = triple["object"]["value"]
                triples.append((subject, predicate, obj))
            except KeyError as e:
                print(f"Error extracting triple: {e}")
                print("Offending triple:", triple)
    
    # Recursively call the function on child nodes
    for key, value in parse_tree.items():
        if isinstance(value, dict):
            extract_triples(value)
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    extract_triples(item)

# Read the CSV file and process each row
with open(input_csv_file, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # Check if there is content in the second column and it doesn't contain the error message
        if len(row) > 1 and row[1] and "Error parsing the query" not in row[1]:
            # Extract triples from the content of the second column
            content = row[1]
            try:
                parse_tree = json.loads(content)
                extract_triples(parse_tree)
            except json.JSONDecodeError:
                # Handle invalid JSON if needed
                pass

# Write the extracted triples to the output CSV file
with open(output_csv_file, 'w', newline='') as csvfile:
    fieldnames = ['subject', 'predicate', 'object']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()

    # Write the triples
    for triple in triples:
        writer.writerow({'subject': triple[0], 'predicate': triple[1], 'object': triple[2]})

print("Triples extracted and written to", output_csv_file)


In [1]:
# Define the input and output file names
input_file = 'find_bug_triples.csv'
output_file = 'filtered_data.csv'

# Open the input data file for reading and the output file for writing
with open(input_file, 'r') as input_file, open(output_file, 'w') as output_file:
    for line in input_file:
        # Split the line into components using tab (',') as the delimiter
        components = line.strip().split(',')

        # Check if the line has exactly three components (subject, predicate, object)
        if len(components) == 3:
            # Write complete triples to the output file
            output_file.write(line)


triple patterns are extracted from the queries, and schema elements are then isolated from these triples. Entities that start with 'var' or 'g_' (indicating a blank node as per sparqljs parser), as well as those containing the substring "nonsensical", are excluded, ensuring only unique entities are considered. 

In [1]:
import csv

input_file_name = 'filtered_data.csv'
output_file_name = 'extracted_entities.csv'

subjects_and_objects = set()  # Using a set to store unique entities

with open(input_file_name, mode ='r') as input_file:
    csvReader = csv.reader(input_file)
    
    # Skip header if there's any
    next(csvReader, None)

    for row in csvReader:
        # Assuming that the subject is the first element and object is the third element in the row
        subject, predicate, object_ = row

        if not subject.startswith('var') and not subject.startswith('g_') and 'nonsensical' not in subject:
            subjects_and_objects.add(subject)
            
        if not object_.startswith('var') and not object_.startswith('g_') and 'nonsensical' not in object_:
            subjects_and_objects.add(object_)

# Write the extracted subjects and objects to an output CSV file
with open(output_file_name, mode ='w', newline='') as output_file:
    csvWriter = csv.writer(output_file)
    
    # Writing header
    csvWriter.writerow(['Entity'])
    
    # Writing data
    for entity in subjects_and_objects:
        csvWriter.writerow([entity])

print(f"Extracted entities have been saved to {output_file_name}")


Extracted entities have been saved to extracted_entities.csv


In [2]:
import pandas as pd

# Set the maximum column width to None to display the entire content of each cell
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('extracted_entities.csv', lineterminator='\n', dtype=str, header=None)

print(len(df))
df.head(100)

30920


,0
0,Entity\r
1,\r
2,http://purl.org/goodrelations/v1#Offering\r
3,SLC9A4\r
4,http://bio2rdf.org/kegg:K03908\r
...,...
95,http://bio2rdf.org/wormbase:WBGene00198889\r
96,http://dbpedia.org/resource/Nortriptyline\r
97,SUCLG1\r
98,http://bio2rdf.org/go.ref_vocabulary:Resource\r


In [ ]:
# Print the clusters and the entities belonging to each cluster
for i in range(num_clusters):
    print(f"Cluster {i}:")
    print(df[df['cluster'] == i]['Entity'])


# query type of entities 

In [ ]:
!pip install SPARQLWrapper


# get types but if the entity is a type return itself and other types

In [7]:
import csv
import re
from SPARQLWrapper import SPARQLWrapper, JSON
 
sparql_endpoint = "http://bio2rdf.org/sparql"

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)

def is_url(string):
    url_regex = re.compile(
        r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    )
    return re.match(url_regex, string) is not None

def execute_query_and_write_results(entity, query, is_entity_url):
    try:
        sparql.setQuery(query)
        results = sparql.query().convert()
        types = set()
        has_results = bool(results["results"]["bindings"])  # Check if the result set is not empty

        for result in results["results"]["bindings"]:
            type_value = result.get("type", {}).get("value")
            
            if type_value:
                if type_value in ["http://www.w3.org/2002/07/owl#Class", "http://www.w3.org/2000/01/rdf-schema#Class"]:
                    types.add(entity)
                else:
                    types.add(type_value)
 
        if has_results and not is_entity_url:
            titles_file.write(entity + '\n')

        for type_value in types:
            types_file.write(type_value + '\n')

        if not types:
            without_type_file.write(entity + '\n')

    except Exception as e:
        print(f"An error occurred while processing the entity {entity}: {e}")

with open('extracted_entities.csv', mode ='r') as file, \
     open('types1.txt', 'w') as types_file, \
     open('titles1.txt', 'w') as titles_file, \
     open('without_type1.txt', 'w') as without_type_file:

    csv_reader = csv.reader(file)
    for row in csv_reader:
        entity = row[0].strip()
        is_entity_url = is_url(entity)

        if is_entity_url:
            query = f"""
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX dcterms: <http://purl.org/dc/terms/>

                SELECT DISTINCT ?type  
                WHERE {{
                    <{entity}> rdf:type ?type .
                                    }}
            """
        else:
            query = f"""
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX dcterms: <http://purl.org/dc/terms/>
                SELECT ?type 
                WHERE {{ 
                       ?s dcterms:title "{entity}" ;
                                rdf:type ?type .
                }}
            """

        execute_query_and_write_results(entity, query, is_entity_url)
        

30918 entities reduced to 1030 types

## Get properties

In [ ]:
import csv
from SPARQLWrapper import SPARQLWrapper, JSON

# List of graphs
graphs = [
"http://bio2rdf.org/sgd_resource:bio2rdf.dataset.sgd.R3",
"http://bio2rdf.org/taxonomy_resource:bio2rdf.dataset.taxonomy.R3",
"http://bio2rdf.org/homologene_resource:bio2rdf.dataset.homologene.R3",
"http://bio2rdf.org/interpro_resource:bio2rdf.dataset.interpro.R3",
"http://bio2rdf.org/bioportal_resource:bio2rdf.dataset.bioportal.R3",
"http://bio2rdf.org/clinicaltrials_resource:bio2rdf.dataset.clinicaltrials.R3",
"http://bio2rdf.org/kegg_resource:bio2rdf.dataset.kegg.R3",
"http://bio2rdf.org/pharmgkb_resource:bio2rdf.dataset.pharmgkb.R3",
"http://bio2rdf.org/hgnc_resource:bio2rdf.dataset.hgnc.R3",
"http://bio2rdf.org/mesh_resource:bio2rdf.dataset.mesh.R3",
"http://bio2rdf.org/omim_resource:bio2rdf.dataset.omim.R3",
"http://bio2rdf.org/sider_resource:bio2rdf.dataset.sider.R3",
"http://bio2rdf.org/apo_resource:bio2rdf.dataset.apo.R3",
"http://bio2rdf.org/ctd_resource:bio2rdf.dataset.ctd.R3",
"http://bio2rdf.org/go_resource:bio2rdf.dataset.go.R3",
"http://bio2rdf.org/hp_resource:bio2rdf.dataset.hp.R3",
"http://bio2rdf.org/drugbank_resource:bio2rdf.dataset.drugbank.R3",
"http://bio2rdf.org/mgi_resource:bio2rdf.dataset.mgi.R3",
"http://bio2rdf.org/goa_resource:bio2rdf.dataset.goa.R3",
"http://bio2rdf.org/ndc_resource:bio2rdf.dataset.ndc.R3",
"http://bio2rdf.org/wormbase_resource:bio2rdf.dataset.wormbase.R3",
"http://bio2rdf.org/lsr_resource:bio2rdf.dataset.lsr.R3",
"http://bio2rdf.org/affymetrix_resource:bio2rdf.dataset.affymetrix.R3",
"http://bio2rdf.org/ncbigene_resource:bio2rdf.dataset.ncbigene.R3",
"http://bio2rdf.org/eco_resource:bio2rdf.dataset.eco.R3",
"http://bio2rdf.org/irefindex_resource:bio2rdf.dataset.irefindex.R3"]

# Function to check if predicate exists in a specific graph of Bio2RDF
def check_predicate_in_graph(predicate, graph):
    sparql = SPARQLWrapper("https://bio2rdf.org/sparql")
    query = f"ASK FROM <{graph}> {{ ?s <{predicate}> ?o }}"
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        results = sparql.query().convert()
        return results['boolean']
    except Exception as e:
        print(f"Error querying predicate {predicate} in graph {graph}: {e}")
        return None

# Lists to hold predicates based on existence in Bio2RDF and errors
exists = []
does_not_exist = []
errors = []

# Read predicates from CSV file and check their existence in each Bio2RDF graph
with open('Logs_properties.csv', mode='r') as file:
    csvFile = csv.reader(file)
    for predicate in csvFile:
        predicate_found = False
        for graph in graphs:
            result = check_predicate_in_graph(predicate[0], graph)
            if result is True:
                exists.append(predicate[0])
                predicate_found = True
                break
            elif result is None:
                errors.append(predicate[0])
                break
        if not predicate_found and predicate[0] not in errors:
            does_not_exist.append(predicate[0])

# Write predicates that exist in Bio2RDF to a file
with open('10_Logs_properties.csv', 'w') as file:
    for predicate in exists:
        file.write(predicate + '\n')

# Write predicates that do not exist in Bio2RDF to a file
with open('10_p_not_in_bio2rdf.txt', 'w') as file:
    for predicate in does_not_exist:
        file.write(predicate + '\n')

# Print or log predicates that caused errors
print("Predicates that caused errors:")
for predicate in errors:
    print(predicate)


# Get Schema vocabularies from KG using queries

In [171]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
import csv

# List of dataset files
dataset_files = [
"http://bio2rdf.org/sgd_resource:bio2rdf.dataset.sgd.R3",
"http://bio2rdf.org/taxonomy_resource:bio2rdf.dataset.taxonomy.R3",
"http://bio2rdf.org/homologene_resource:bio2rdf.dataset.homologene.R3",
"http://bio2rdf.org/interpro_resource:bio2rdf.dataset.interpro.R3",
"http://bio2rdf.org/bioportal_resource:bio2rdf.dataset.bioportal.R3",
"http://bio2rdf.org/clinicaltrials_resource:bio2rdf.dataset.clinicaltrials.R3",
"http://bio2rdf.org/kegg_resource:bio2rdf.dataset.kegg.R3",
"http://bio2rdf.org/pharmgkb_resource:bio2rdf.dataset.pharmgkb.R3",
"http://bio2rdf.org/hgnc_resource:bio2rdf.dataset.hgnc.R3",
"http://bio2rdf.org/mesh_resource:bio2rdf.dataset.mesh.R3",
"http://bio2rdf.org/omim_resource:bio2rdf.dataset.omim.R3",
"http://bio2rdf.org/sider_resource:bio2rdf.dataset.sider.R3",
"http://bio2rdf.org/apo_resource:bio2rdf.dataset.apo.R3",
"http://bio2rdf.org/ctd_resource:bio2rdf.dataset.ctd.R3",
"http://bio2rdf.org/go_resource:bio2rdf.dataset.go.R3",
"http://bio2rdf.org/hp_resource:bio2rdf.dataset.hp.R3",
"http://bio2rdf.org/drugbank_resource:bio2rdf.dataset.drugbank.R3",
"http://bio2rdf.org/mgi_resource:bio2rdf.dataset.mgi.R3",
"http://bio2rdf.org/goa_resource:bio2rdf.dataset.goa.R3",
"http://bio2rdf.org/ndc_resource:bio2rdf.dataset.ndc.R3",
"http://bio2rdf.org/wormbase_resource:bio2rdf.dataset.wormbase.R3",
"http://bio2rdf.org/lsr_resource:bio2rdf.dataset.lsr.R3",
"http://bio2rdf.org/affymetrix_resource:bio2rdf.dataset.affymetrix.R3",
"http://bio2rdf.org/ncbigene_resource:bio2rdf.dataset.ncbigene.R3",
"http://bio2rdf.org/eco_resource:bio2rdf.dataset.eco.R3",
"http://bio2rdf.org/irefindex_resource:bio2rdf.dataset.irefindex.R3"]

# Replace `your_sparql_endpoint` with the actual SPARQL endpoint URL
sparql_endpoint = "http://bio2rdf.org/sparql"
def execute_query(sparql_query):
    sparql = SPARQLWrapper(sparql_endpoint)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    try:
        results = sparql.query().convert()
        print(f"Query returned {len(results['results']['bindings'])} results.")  # Diagnostic print
        return results
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# This will hold all distinct class types and predicates
classes_set = set()
predicates_set = set()

# Function to write results to CSV
def write_results_to_csv(file_name, results_set):
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for item in results_set:
            writer.writerow([item])
        print(f"Wrote {len(results_set)} items to {file_name}")  # Diagnostic print

# Loop through dataset files to construct the schema
for file_url in dataset_files:
    # dataset_uri = extract_dataset_uri(file_url)
    # if dataset_uri:
        print(f"Processing {file_url}")
        # Query to find all distinct types
        query_types = f"""
        SELECT DISTINCT ?type
        FROM <{file_url}>
        WHERE {{ ?s a ?type }}
        """
        results_types = execute_query(query_types)
        for result in results_types["results"]["bindings"]:
            classes_set.add(result["type"]["value"])
        
        # Query to find all distinct predicates
        query_predicates = f"""
        SELECT DISTINCT ?p
        FROM <{file_url}>
        WHERE {{ ?s ?p ?o }}
        """
        results_predicates = execute_query(query_predicates)
        for result in results_predicates["results"]["bindings"]:
            predicates_set.add(result["p"]["value"])
        
# Write the results to CSV files
write_results_to_csv('10_Classes_schema.csv', classes_set)
write_results_to_csv('10_Predicates_schema.csv', predicates_set)

Processing http://bio2rdf.org/sgd_resource:bio2rdf.dataset.sgd.R3
Query returned 92 results.
Query returned 74 results.
Processing http://bio2rdf.org/taxonomy_resource:bio2rdf.dataset.taxonomy.R3
Query returned 11 results.
Query returned 51 results.
Processing http://bio2rdf.org/homologene_resource:bio2rdf.dataset.homologene.R3
Query returned 12 results.
Query returned 24 results.
Processing http://bio2rdf.org/interpro_resource:bio2rdf.dataset.interpro.R3
Query returned 2 results.
Query returned 13 results.
Processing http://bio2rdf.org/bioportal_resource:bio2rdf.dataset.bioportal.R3
Query returned 5 results.
Query returned 35 results.
Processing http://bio2rdf.org/clinicaltrials_resource:bio2rdf.dataset.clinicaltrials.R3
Query returned 65 results.
Query returned 168 results.
Processing http://bio2rdf.org/kegg_resource:bio2rdf.dataset.kegg.R3
Query returned 77 results.
Query returned 150 results.
Processing http://bio2rdf.org/pharmgkb_resource:bio2rdf.dataset.pharmgkb.R3
Query returned

# Get superclasses as well

In [175]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# Initialize the SPARQL wrapper with the endpoint URL
sparql = SPARQLWrapper("http://bio2rdf.org/sparql")

# Function to get the superclass of a given class
def get_superclass(class_uri):
    query = """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT DISTINCT ?superclass
    WHERE {
      <""" + class_uri + """> rdfs:subClassOf ?superclass .
    }
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    # Extract the superclass URIs from the query results
    superclasses = [result['superclass']['value'] for result in results["results"]["bindings"]]
    # print("s")
    # print("superclasses", superclasses)
    return superclasses

# Read the classes from the CSV file
classes_df = pd.read_csv('10_superclasses.csv')

# This assumes that the first column contains the class URIs
class_uris = classes_df.iloc[:, 0].unique()

# Get the superclasses for each class URI
superclasses_list = []
for class_uri in class_uris:
    superclasses = get_superclass(class_uri)
    superclasses_list.extend(superclasses)

# Remove duplicates and write to CSV
superclasses_df = pd.DataFrame(list(set(superclasses_list)), columns=['Superclass'])
superclasses_df.to_csv('10_sup_superclasses.csv', index=False)


# Get subclasses as well

In [ ]:
!pip install SPARQLWrapper

In [14]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# Initialize the SPARQL wrapper with the endpoint URL
sparql = SPARQLWrapper("http://bio2rdf.org/sparql")

# Function to get the superclass of a given class
def get_superclass(class_uri):
    query = """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT DISTINCT ?subclass
    WHERE {
      ?subclass rdfs:subClassOf <""" + class_uri + """> .
    }
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    # Extract the superclass URIs from the query results
    superclasses = [result['subclass']['value'] for result in results["results"]["bindings"]]
    # print("s")
    # print("superclasses", superclasses)
    return superclasses

# Read the classes from the CSV file
classes_df = pd.read_csv('10_subclass.csv')

# This assumes that the first column contains the class URIs
class_uris = classes_df.iloc[:, 0].unique()

# Get the superclasses for each class URI
superclasses_list = []
for class_uri in class_uris:
    superclasses = get_superclass(class_uri)
    superclasses_list.extend(superclasses)

# Remove duplicates and write to CSV
superclasses_df = pd.DataFrame(list(set(superclasses_list)), columns=['Superclass'])
superclasses_df.to_csv('10_sub_subclass.csv', index=False)


s
superclasses ['http://bio2rdf.org/wormbase_vocabulary:Change_of_expression_level-Regulatory-Interaction', 'http://bio2rdf.org/wormbase_vocabulary:Regulatory-Interaction', 'http://bio2rdf.org/wormbase_vocabulary:Change_of_localization-Regulatory-Interaction']
s
superclasses []
s
superclasses []


 ##  clean schema classes [KG]

In [172]:
import csv

# Define the input and output filenames
input_filename = '10_Classes_schema.csv'
output_filename = '10_bio2rdf_spesific_Classes_schema.csv'
count = 0
# Open the input file for reading and the output file for writing
with open(input_filename, mode='r', newline='', encoding='utf-8') as infile, \
        open(output_filename, mode='w', newline='', encoding='utf-8') as outfile:

    # Create reader and writer objects
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Iterate over each row in the input CSV
    for row in reader:
        # Check each cell in the row
        for value in row:
            # If the cell contains '_vocabulary' or contains 'http:' without ':', write the row to the output CSV
#             or ('http:' in value and ':' not in value.replace('http:', ''))
            if '_vocabulary' in value :
                writer.writerow(row)
                count = count + 1
                break  # We write the row once if it meets the condition and then move to the next row

print(f"Rows have been written to {output_filename} as per the conditions.")
print("count is:", count )

Rows have been written to 10_bio2rdf_spesific_Classes_schema.csv as per the conditions.
count is: 527



## get the entities that are in kg schema but not in queries 

In [ ]:
 import csv

# Step 1: Read entities from `u_types1_labels.csv`
with open("10_bio2rdf_total_schema_elements.txt", "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    entities = [row[0] for row in csv_reader]  # assuming the entity is in the first column

# Step 2: Check if they exist in `1u-labels-u-kg-types1.txt`
with open("types20.csv", "r") as txt_file:
    txt_content = txt_file.read().splitlines()

entities_in_kg = [entity for entity in entities if entity in txt_content]
entities_not_in_kg = [entity for entity in entities if entity not in txt_content]

# Step 3: Write the existing entities to `EntitiesInQueriesExist_in_KG_schema.csv`
with open("typesInQueriesExist_in_KG_schema.csv", "w", newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    for entity in entities_in_kg:
        csv_writer.writerow([entity])

# Step 4: Print the count and the entities that don't exist
print(f"Count of entities in queries that exist in KG schema: {len(entities_not_in_kg)}")
print("\nEntities that don't exist in KG schema:")
for entity in entities_not_in_kg:
    print(entity)



## get the entities that are in queries but not in kg schema

In [9]:
import csv

# Step 1: Read entities from `u_types1_labels.csv`
with open("types20.csv", "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    entities = [row[0] for row in csv_reader]  # assuming the entity is in the first column

# Step 2: Check if they exist in `1u-labels-u-kg-types1.txt`
with open( "10_sup_bio2rdf_spesific_Classes_schema.csv" , "r") as txt_file:
    txt_content = txt_file.read().splitlines()

entities_in_kg = [entity for entity in entities if entity in txt_content]
entities_not_in_kg = [entity for entity in entities if entity not in txt_content]

# Step 3: Write the existing entities to `EntitiesInQueriesExist_in_KG_schema.csv`
with open("queries_types_valid_KG_schema.csv", "w", newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    for entity in entities_in_kg:
        csv_writer.writerow([entity])

# Step 4: Print the count and the entities that don't exist
print(f"Count of entities in queries that not exist in KG schema: {len(entities_not_in_kg)}")
print("\nEntities that don't exist in KG schema:")
for entity in entities_not_in_kg:
    print(entity)


Count of entities in queries that not exist in KG schema: 4

Entities that don't exist in KG schema:
http://bio2rdf.org/hgnc_vocabulary:Status
http://bio2rdf.org/drugbank_vocabulary:7cab3885cdbcb9df8c405e9c9ad10732
http://bio2rdf.org/hgnc_vocabulary:Approved
http://bio2rdf.org/atmo_vocabulary:Resource


 ##  clean properties extracted from queries [Logs_properties.csv]

In [10]:
import csv

# Step 1: Read entities from `u_types1_labels.csv`
with open("Logs_properties.csv", "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    entities = [row[0] for row in csv_reader]  # assuming the entity is in the first column

# Step 2: Check if they exist in `1u-labels-u-kg-types1.txt`
with open( "10_Predicates_schema.csv" , "r") as txt_file:
    txt_content = txt_file.read().splitlines()

entities_in_kg = [entity for entity in entities if entity in txt_content]
entities_not_in_kg = [entity for entity in entities if entity not in txt_content]

# Step 3: Write the existing entities to `EntitiesInQueriesExist_in_KG_schema.csv`
with open("queries_valid_properties_KG_schema.csv", "w", newline='') as csv_output:
    csv_writer = csv.writer(csv_output)
    for entity in entities_in_kg:
        csv_writer.writerow([entity])

# Step 4: Print the count and the entities that don't exist
print(f"Count of properties in queries that not exist in KG schema: {len(entities_not_in_kg)}")

print("\n properties that don't exist in KG schema:")
for entity in entities_not_in_kg:
    print(entity)


Count of properties in queries that not exist in KG schema: 198

 properties that don't exist in KG schema:
http://www.openlinksw.com/schemas/virtrdf#qmvftXmlIndex
http://purl.org/goodrelations/v1#includes
http://www.w3.org/ns/dcat#downloadURL
http://semanticscience.org/resource/SIO_000205
http://www.openlinksw.com/schemas/virtrdf#qmfSuperFormats
http://www.openlinksw.com/schemas/virtrdf#qmfCmpFuncName
http://www.openlinksw.com/schemas/virtrdf#qmvcAlias
http://www.openlinksw.com/schemas/virtrdf#qmfMapsOnlyNullToNull
http://www.openlinksw.com/schemas/virtrdf#qsMatchingFlags
http://www.w3.org/ns/dcat#theme
http://www.w3.org/2000/01/rdf-schema#domain
http://www.openlinksw.com/schemas/virtrdf#qmfSparqlEbvTmpl
http://www.openlinksw.com/schemas/virtrdf#qmfIidOfShortTmpl
http://purl.org/dc/terms/accrualPeriodicity
http://purl.org/dc/terms/hasPart
http://rdfs.org/ns/void#propertyPartition
http://www.openlinksw.com/schemas/virtrdf#qmfShortTmpl
http://semanticscience.org/resource/SIO_000008
http

# Get total schema element of queries

In [11]:
# Set to hold the unique elements from both files
unique_elements = set()

# Read the property_types.txt file and add its contents to the set
with open('queries_valid_properties_KG_schema.csv', 'r') as file:
    for line in file:
        unique_elements.add(line.strip())  # strip() removes any leading/trailing whitespace including newline characters

# Read the types1.csv file and add its contents to the set
with open('queries_types_valid_KG_schema.csv', 'r') as file:
    for line in file:
        unique_elements.add(line.strip())

# Write the unique elements to a new file
with open('Logs_total_schema_elements.txt', 'w') as file:
    for element in unique_elements:
        file.write(element + '\n')

print(f"{len(unique_elements)} unique elements have been written to unique_elements.txt")


1606 unique elements have been written to unique_elements.txt


# Get total schema element count of bio2rdf

In [176]:
# Set to hold the unique elements from both files
unique_elements = set()

# Read the property_types.txt file and add its contents to the set
with open('10_sup_bio2rdf_spesific_Classes_schema.csv', 'r') as file:
    for line in file:
        unique_elements.add(line.strip())  # strip() removes any leading/trailing whitespace including newline characters

# Read the types1.csv file and add its contents to the set
with open('10_Predicates_schema.csv', 'r') as file:
    for line in file:
        unique_elements.add(line.strip())

# Write the unique elements to a new file
with open('10_bio2rdf_total_schema_elements.txt', 'w') as file:
    for element in unique_elements:
        file.write(element + '\n')

print(f"{len(unique_elements)} unique elements have been written to unique_elements.txt")


1635 unique elements have been written to unique_elements.txt


In [5]:
import csv

# Step 1: Read entities from `u_types1_labels.csv`
with open("Logs_total_schema_elements.txt" , "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    entities = [row[0] for row in csv_reader] # assuming the entity is in the first column

# Step 2: Check if they exist in `1u-labels-u-kg-types1.txt`
with open( "10_bio2rdf_total_schema_elements.txt"  , "r") as txt_file:
    txt_content = txt_file.read().splitlines()

entities_in_kg = [entity for entity in entities if entity in txt_content]
entities_not_in_kg = [entity for entity in entities if entity not in txt_content]


# Step 4: Print the count and the entities that don't exist
print(f"Count of entities in queries that exist in KG schema: {len(entities_not_in_kg)}")
print("\nEntities that don't exist in KG schema:")
for entity in entities_not_in_kg:
    print(entity)


Count of entities in queries that exist in KG schema: 202

Entities that don't exist in KG schema:
http://rdfs.org/ns/void#subjectsTarget
http://rdfs.org/ns/void#property
http://www.openlinksw.com/schemas/virtrdf#qmfDatatypeOfShortTmpl
http://www.openlinksw.com/schemas/virtrdf#qmvFormat
http://purl.org/dc/terms/modified
http://www.w3.org/2002/07/owl#priorVersion
http://purl.org/dc/terms/hasPart
http://bio2rdf.org/hgnc_vocabulary:Approved
http://www.bigdata.com/rdf#/features/KB/Namespace
http://www.openlinksw.com/schemas/virtrdf#qmfUriOfShortTmpl
http://www.w3.org/ns/sparql-service-description#resultFormat
http://www.openlinksw.com/schemas/virtrdf#qmfExistingShortOfUriTmpl
http://purl.org/goodrelations/v1#legalName
http://purl.org/goodrelations/v1#availableDeliveryMethods
http://www.openlinksw.com/schemas/virtrdf#qmfColumnCount
http://rdfs.org/ns/void#classes
http://www.openlinksw.com/schemas/virtrdf#qmfIsBijection
http://www.w3.org/1999/02/22-rdf-syntax-ns#_4
http://purl.org/ontology/w

In [26]:
import csv

# Read entities from `bio2rdf_total_schema_elements.txt`
with open("bio2rdf_total_schema_elements.txt", "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    entities = [row[0] for row in csv_reader]  # assuming the entity is in the first column

# Check if they exist in `Logs_total_schema_elements.txt`
with open("Logs_total_schema_elements.txt", "r") as txt_file:
    txt_content = txt_file.read().splitlines()

# Determine which entities are not in the knowledge graph
entities_not_in_kg = [entity for entity in entities if entity not in txt_content]

# Step 4: Print the count and the entities that don't exist
# We will sort the entities based on whether they contain '_vocabulary:Resource' or '_vocabulary'
resource_entities = [entity for entity in entities_not_in_kg if '_vocabulary:Resource' in entity]
vocabulary_entities = [entity for entity in entities_not_in_kg if '_vocabulary' in entity and entity not in resource_entities]
other_entities = [entity for entity in entities_not_in_kg if entity not in resource_entities and entity not in vocabulary_entities]

# Now we combine the lists, keeping the desired order
sorted_entities_not_in_kg = vocabulary_entities + resource_entities + other_entities

print(f"Count of entities not in KG schema: {len(sorted_entities_not_in_kg)}")
print("\nEntities that don't exist in KG schema:")
for entity in sorted_entities_not_in_kg:
    print(entity)
# 10_Subclass_all.csv

Count of entities not in KG schema: 54

Entities that don't exist in KG schema:
http://bio2rdf.org/clinicaltrials_vocabulary:limitations-and-caveats
http://bio2rdf.org/clinicaltrials_vocabulary:ci-upper-limit
http://bio2rdf.org/clinicaltrials_vocabulary:lower-limit
http://bio2rdf.org/obo_vocabulary:Entity
http://bio2rdf.org/kegg_vocabulary:Reversible-Reaction
http://bio2rdf.org/clinicaltrials_vocabulary:ci-upper-limit-na-comment
http://bio2rdf.org/clinicaltrials_vocabulary:upper-limit
http://bio2rdf.org/clinicaltrials_vocabulary:ci-lower-limit
http://bio2rdf.org/kegg_vocabulary:Irreversible-Reaction
http://bio2rdf.org/broad-lincrna_vocabulary:Resource
http://bio2rdf.org/ed_vocabulary:Resource
http://bio2rdf.org/ligandbox_vocabulary:Resource
http://bio2rdf.org/ncbi-proteinid_vocabulary:Resource
http://bio2rdf.org/solgenomics_vocabulary:Resource
http://bio2rdf.org/luo_lincrna_vocabulary:Resource
http://bio2rdf.org/signalp_vocabulary:Resource
http://bio2rdf.org/utoronto_vocabulary:Resourc

In [18]:
import csv

# Read entities from `10_Subclass_all.csv`
entities = []
with open("10_Subclass_all.csv", "r") as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        if row:  # Check if row is not empty
            entities.append(row[0])  # Assuming the entity is in the first column

# Check if they exist in `10_bio2rdf_total_schema_elements.txt`
with open("10_bio2rdf_total_schema_elements.txt", "r") as txt_file:
    txt_content = txt_file.read().splitlines()

# Determine which entities are not in the knowledge graph
entities_not_in_kg = [entity for entity in entities if entity not in txt_content]

print(f"Count of entities not in KG schema: {len(entities_not_in_kg)}")
print("\nEntities that don't exist in KG schema:")
for entity in entities_not_in_kg:
    print(entity)


Count of entities not in KG schema: 0

Entities that don't exist in KG schema:



Total Schema Elements (TSE): Count of all distinct classes, predicates in Bio2rdf dataset.

Used Schema Elements (USE): Count of all distinct classes, predicates  that are used in user sparql queries.

Schema Coverage (SC) = ( USE /  TSE) × 100 
